# Final Project: Garbage Detection

## Import packages

In [1]:
import os

import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder

from garbage_detector.application import Application
from garbage_detector.models import train
from garbage_detector.models.util import load_states_, save_states
from garbage_detector.models.benchmark import (DatasetGenerator, DatasetSize,
                                              benchmark)
from garbage_detector.models.classification.cnn import CNNModel
from garbage_detector.models.detection.fasterrcnn import (
    FasterRCNNModel, FasterRCNNModelGenerator)
from garbage_detector.models.detection.fasterrcnn.custom import (
    mobile_net_backbone, resnet50_backbone)
from garbage_detector.models.detection.fasterrcnn.data import (
    GarbageDetectionDataset, collate_fn)
from garbage_detector.util.io import CSVReader, CSVWriter, get_project_root_dir


## Setup of computing resources

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 0, 'pin_memory': True} if torch.cuda.is_available() else {
}

## CNN Garbage Classification Benchmark

##### Setup

In [ ]:
cnn_batch_size = 64
cnn_test_batch_size = 100
cnn_epochs = 10
cnn_lr = 1.0
cnn_gamma = 0.7
cnn_step_size = 1

cnn_benchmark_set_sizes = {
    DatasetSize.SMALL: 2500,
    DatasetSize.MEDIUM: 7500,
    DatasetSize.LARGE: 14819
}

class_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2,
                           saturation=0.1, hue=0.1),
    transforms.RandomAffine(degrees=40, translate=None,
                            scale=(1, 2), shear=15),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

cnn_writer = CSVWriter(os.path.join('results', 'classification',
                                    'benchmark.csv'), ['model', 'dataset', 'accuracy'])
data_path = os.path.join(get_project_root_dir(), 'data',
                         'kaggle', 'garbage_classification', 'normal')

class_dataset = ImageFolder(data_path, class_transform)

class_dataset_gen = DatasetGenerator(
    class_dataset, kwargs, cnn_benchmark_set_sizes, cnn_batch_size, cnn_test_batch_size)

cnn_model_list = [CNNModel.MOBILE_NET, CNNModel.RESNET_50,
                  CNNModel.GOOGLE_NET, CNNModel.EFFICIENT_NET]


##### Run Benchmark

In [ ]:
criterion = nn.CrossEntropyLoss()
cnn_out_features = len(class_dataset.classes)

cnn_results = benchmark.run_benchmark(cnn_model_list, device, cnn_lr, cnn_gamma, cnn_step_size,
                                      cnn_out_features, criterion, class_dataset_gen.train_loaders, class_dataset_gen.test_loader, cnn_epochs)
cnn_writer.write_header()
cnn_writer.write_rows(cnn_results)
cnn_writer.close()


##### Show results

In [3]:
cnn_res_reader = CSVReader(os.path.join(
    'results', 'classification', 'benchmark.csv'))
cnn_res_reader()

,model,dataset,accuracy
0,cnn_mobilenet,small,85.04
1,cnn_mobilenet,medium,88.31
2,cnn_mobilenet,large,89.45
3,cnn_resnet50,small,86.61
4,cnn_resnet50,medium,90.09
5,cnn_resnet50,large,91.65
6,cnn_googlenet,small,84.09
7,cnn_googlenet,medium,88.22
8,cnn_googlenet,large,89.23
9,cnn_efficientnet,small,86.93


## Faster RCCN Garbage Detection Benchmark

##### Setup

In [ ]:
fasterrcnn_batch_size = 2
fasterrcnn_test_batch_size = 2
fasterrcnn_epochs = 10
fasterrcnn_lr = 0.005
fasterrcnn_gamma = 0.1
fasterrcnn_step_size = 3

fasterrcnn_benchmark_set_sizes = {
    DatasetSize.SMALL: 200,
    DatasetSize.MEDIUM: 600,
    DatasetSize.LARGE: 1100
}

detect_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ConvertImageDtype(torch.float)
])

fasterrcnn_writer = CSVWriter(os.path.join('results', 'detection', 'benchmark.csv'), [
    'model', 'dataset', 'map', 'map_50', 'map_75', 'mar_100'])
detect_dataset = GarbageDetectionDataset(os.path.join(
    'data', 'kaggle', 'garbage_classification', 'annotated'), detect_transform)

fasterrcnn_out_features = len(detect_dataset.get_classes()) + 1
detect_dataset_gen = DatasetGenerator(detect_dataset, kwargs, fasterrcnn_benchmark_set_sizes, fasterrcnn_batch_size,
                                      fasterrcnn_test_batch_size, collate_fn=collate_fn)

fasterrcnn_model_list = [FasterRCNNModel.RESNET_50, FasterRCNNModel.MOBILE_NET, FasterRCNNModel.CUSTOM_RESNET_50,
                         FasterRCNNModel.CUSTOM_MOBILE_NET]


##### Run Benchmark

In [ ]:
results = benchmark.run_benchmark(fasterrcnn_model_list, device, fasterrcnn_lr, fasterrcnn_gamma, fasterrcnn_step_size,
                                  fasterrcnn_out_features, None, detect_dataset_gen.train_loaders, detect_dataset_gen.test_loader, fasterrcnn_epochs, True)
fasterrcnn_writer.write_header()
fasterrcnn_writer.write_rows(results)
fasterrcnn_writer.close()

##### Show Results

In [4]:
fasterrcnn_res_reader = CSVReader(os.path.join(
    'results', 'detection', 'benchmark.csv'))
fasterrcnn_res_reader()

,model,dataset,map,map_50,map_75,mar_100
0,fasterrcnn_resnet50,small,48.0,61.4,57.6,74.0
1,fasterrcnn_resnet50,medium,65.6,79.7,75.2,83.1
2,fasterrcnn_resnet50,large,84.4,95.1,93.3,90.9
3,fasterrcnn_mobilenet,small,54.4,69.1,64.3,71.8
4,fasterrcnn_mobilenet,medium,79.5,92.9,89.0,85.2
5,fasterrcnn_mobilenet,large,89.6,98.4,97.0,92.4
6,fasterrcnn_custom_resnet50,small,0.0,0.0,0.0,0.4
7,fasterrcnn_custom_resnet50,medium,0.6,3.2,0.0,4.5
8,fasterrcnn_custom_resnet50,large,1.4,6.4,0.2,6.7
9,fasterrcnn_custom_mobilenet,small,6.6,15.9,3.7,15.7


## Garbage Detection Demo

##### Setup

In [ ]:
fasterrcnn_batch_size = 2
fasterrcnn_test_batch_size = 2
fasterrcnn_epochs = 10
fasterrcnn_lr = 0.005
fasterrcnn_gamma = 0.1
fasterrcnn_step_size = 3

detect_dataset = GarbageDetectionDataset(os.path.join(
    'data', 'kaggle', 'garbage_classification', 'annotated'), None)

fasterrcnn_out_features = len(detect_dataset.get_classes()) + 1


fasterrcnn_model_list = [FasterRCNNModel.RESNET_50, FasterRCNNModel.MOBILE_NET, FasterRCNNModel.CUSTOM_RESNET_50,
                         FasterRCNNModel.CUSTOM_MOBILE_NET]

fasterrcnn_model_gen = FasterRCNNModelGenerator()

fasterrcnn_custom_models = {
    FasterRCNNModel.CUSTOM_RESNET_50: fasterrcnn_model_gen.create_model_with_custom_backbone(resnet50_backbone, fasterrcnn_lr, fasterrcnn_gamma, fasterrcnn_step_size, fasterrcnn_out_features),
    FasterRCNNModel.CUSTOM_MOBILE_NET: fasterrcnn_model_gen.create_model_with_custom_backbone(
        mobile_net_backbone, fasterrcnn_lr, fasterrcnn_gamma, fasterrcnn_step_size, fasterrcnn_out_features)
}

fasterrcnn_models = {m: fasterrcnn_model_gen.create_model(m, fasterrcnn_lr, fasterrcnn_gamma, fasterrcnn_step_size, fasterrcnn_out_features)
                     if 'custom' not in m else fasterrcnn_custom_models[m] for m in fasterrcnn_model_list}


##### Train

In [ ]:
demo_train_epoch = 20

for m in fasterrcnn_models:
    model, optimizer, scheduler = fasterrcnn_models[m]
    load_states_(m, model)
    for epoch in range(0, demo_train_epoch):
        train(model, device, detect_dataset_gen.test_loader, optimizer)
        scheduler.step()
        save_states(m, model, optimizer)


##### Run Application

In [ ]:
app_name = 'FRCNN Garbage Detection'
detection_threshold = 0.6

demo_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

demo_model = fasterrcnn_models[FasterRCNNModel.RESNET_50][0]
demo_model.to(device)
load_states_(FasterRCNNModel.RESNET_50, demo_model, force_download=True)

classes = detect_dataset.get_classes().copy()
classes.insert(0, 'background')
application = Application(app_name, demo_model, device,
                          demo_transform, classes, detection_threshold)
application.run()
